In [1]:
import tensorflow as tf
keras = tf.keras
from keras.layers import SimpleRNN, Dense, Flatten
from keras import Sequential, Input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

2023-08-31 20:52:49.373568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 20:52:52.857021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def gen_time_series(sample_size: int, n_steps: int):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, sample_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(sample_size, n_steps) - 0.5)

    return series[..., np.newaxis].astype(np.float32)

In [3]:
data = gen_time_series(10000, 51)

In [4]:
data[0]

array([[-0.19207342],
       [-0.17525306],
       [-0.1815216 ],
       [-0.22039606],
       [-0.12343376],
       [-0.08581077],
       [ 0.17670955],
       [ 0.4084893 ],
       [ 0.5380089 ],
       [ 0.659116  ],
       [ 0.7142563 ],
       [ 0.58313876],
       [ 0.39349225],
       [ 0.16831467],
       [ 0.06830373],
       [-0.08398375],
       [-0.0411288 ],
       [-0.10121907],
       [-0.12894028],
       [-0.17277212],
       [-0.3488574 ],
       [-0.42891318],
       [-0.5649305 ],
       [-0.6568419 ],
       [-0.62875456],
       [-0.41220278],
       [-0.18285133],
       [ 0.00484179],
       [ 0.19324917],
       [ 0.3237057 ],
       [ 0.36699268],
       [ 0.2913023 ],
       [ 0.30983433],
       [ 0.332467  ],
       [ 0.3181533 ],
       [ 0.47357017],
       [ 0.53559434],
       [ 0.55582786],
       [ 0.462842  ],
       [ 0.22962232],
       [ 0.00897526],
       [-0.23376514],
       [-0.42377013],
       [-0.49744326],
       [-0.51802593],
       [-0

In [5]:
X = data[:, 0:50]
X

array([[[-0.19207342],
        [-0.17525306],
        [-0.1815216 ],
        ...,
        [-0.27118856],
        [-0.24353215],
        [-0.33677945]],

       [[ 0.48854834],
        [ 0.23283596],
        [-0.00730349],
        ...,
        [-0.4530908 ],
        [-0.5089068 ],
        [-0.5753812 ]],

       [[-0.5506837 ],
        [-0.55035186],
        [-0.49812248],
        ...,
        [ 0.3811657 ],
        [ 0.24647099],
        [ 0.15311971]],

       ...,

       [[-0.59032387],
        [-0.6461003 ],
        [-0.6593416 ],
        ...,
        [ 0.6260169 ],
        [ 0.5046025 ],
        [ 0.41829357]],

       [[ 0.20745422],
        [ 0.16865458],
        [ 0.14276183],
        ...,
        [ 0.24837051],
        [ 0.2648372 ],
        [ 0.24920222]],

       [[-0.56470513],
        [-0.29233664],
        [ 0.00693571],
        ...,
        [-0.26561105],
        [-0.24966209],
        [-0.10444074]]], dtype=float32)

In [6]:
y = data[:, -1]
y

array([[-0.29666665],
       [-0.5548032 ],
       [-0.02663113],
       ...,
       [ 0.13580354],
       [ 0.21837582],
       [-0.02939698]], dtype=float32)

In [7]:
X_train, X_val, X_test = X[0:7000, :], X[7000:9000, :], X[9000:10000, :]
X_val.shape

(2000, 50, 1)

In [8]:
y_train, y_val, y_test = y[0:7000, :], y[7000:9000, :], y[9000:10000, :]

In [9]:
y_train.shape

(7000, 1)

In [10]:
fcnn_model = Sequential([
    Flatten(input_shape=[50, 1]),
    Dense(1)
])
fcnn_model.summary()

2023-08-31 20:53:00.871657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-31 20:53:02.095202: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-31 20:53:02.095289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-31 20:53:02.132795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-31 20:53:02.132893: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 51 (204.00 Byte)
Trainable params: 51 (204.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
fcnn_model.compile(loss="mse", optimizer="adam")

In [12]:
fcnn_hist = fcnn_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20


2023-08-31 20:53:05.674506: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3ff40d1330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-31 20:53:05.674584: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2023-08-31 20:53:06.238538: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-31 20:53:06.528970: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-31 20:53:07.436029: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/219 [..............................] - ETA: 13:09 - loss: 0.6989

2023-08-31 20:53:07.908476: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


219/219 [==============================] - 5s 8ms/step - loss: 0.1789 - val_loss: 0.0656
Epoch 2/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0459 - val_loss: 0.0349
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0285 - val_loss: 0.0251
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0216 - val_loss: 0.0201
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0177 - val_loss: 0.0169
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0150 - val_loss: 0.0145
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0130 - val_loss: 0.0126
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0113 - val_loss: 0.0110
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0099 - val_loss: 0.0096
Epoch 10/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0086 - val_loss: 0.0084
Epoch 11/

In [13]:
fcnn_model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 3ms/step - loss: 0.0038


0.0037785314489156008

In [14]:
rnn1_model = Sequential([
    Input((50, 1)),
    SimpleRNN(1)
])
rnn1_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1)                 3         
                                                                 
Total params: 3 (12.00 Byte)
Trainable params: 3 (12.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
rnn1_model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])

In [16]:
rnn1_hist = rnn1_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
219/219 [==============================] - 24s 103ms/step - loss: 0.0181 - accuracy: 0.0000e+00 - val_loss: 0.0143 - val_accuracy: 0.0000e+00
Epoch 2/20
219/219 [==============================] - 22s 102ms/step - loss: 0.0125 - accuracy: 0.0000e+00 - val_loss: 0.0120 - val_accuracy: 0.0000e+00
Epoch 3/20
219/219 [==============================] - 22s 100ms/step - loss: 0.0115 - accuracy: 0.0000e+00 - val_loss: 0.0117 - val_accuracy: 0.0000e+00
Epoch 4/20
219/219 [==============================] - 22s 101ms/step - loss: 0.0113 - accuracy: 0.0000e+00 - val_loss: 0.0116 - val_accuracy: 0.0000e+00
Epoch 5/20
219/219 [==============================] - 22s 102ms/step - loss: 0.0113 - accuracy: 0.0000e+00 - val_loss: 0.0116 - val_accuracy: 0.0000e+00
Epoch 6/20
219/219 [==============================] - 22s 102ms/step - loss: 0.0113 - accuracy: 0.0000e+00 - val_loss: 0.0116 - val_accuracy: 0.0000e+00
Epoch 7/20
219/219 [==============================] - 23s 106ms/step - loss: 0.011

In [17]:
rnn1_model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 15ms/step - loss: 0.0111 - accuracy: 0.0000e+00


[0.011060044169425964, 0.0]

In [18]:
rnn2_model = Sequential([
    Input((50, 1)),
    SimpleRNN(1, return_sequences=True),
    SimpleRNN(1)
])
rnn2_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 50, 1)             3         
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 1)                 3         
                                                                 
Total params: 6 (24.00 Byte)
Trainable params: 6 (24.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
rnn2_model.compile(loss="mse", optimizer="adam")

In [20]:
rnn2_hist = rnn2_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
219/219 [==============================] - 55s 241ms/step - loss: 0.1444 - val_loss: 0.1438
Epoch 2/20
219/219 [==============================] - 53s 243ms/step - loss: 0.1414 - val_loss: 0.1451
Epoch 3/20
219/219 [==============================] - 52s 238ms/step - loss: 0.1403 - val_loss: 0.1442
Epoch 4/20
219/219 [==============================] - 52s 239ms/step - loss: 0.1403 - val_loss: 0.1424
Epoch 5/20
219/219 [==============================] - 52s 236ms/step - loss: 0.1405 - val_loss: 0.1427
Epoch 6/20
219/219 [==============================] - 52s 237ms/step - loss: 0.1406 - val_loss: 0.1439
Epoch 7/20
219/219 [==============================] - 51s 233ms/step - loss: 0.1403 - val_loss: 0.1425
Epoch 8/20
219/219 [==============================] - 50s 229ms/step - loss: 0.1403 - val_loss: 0.1423
Epoch 9/20
219/219 [==============================] - 52s 235ms/step - loss: 0.1407 - val_loss: 0.1422
Epoch 10/20
219/219 [==============================] - 51s 231ms/step - l

In [ ]:
rnn2_model.evaluate(X_test, y_test)

32/32 [==============================] - 1s 23ms/step - loss: 0.0095


0.009496123529970646

In [ ]:
rnn3_model = Sequential([
    Input((50, 1)),
    SimpleRNN(1, return_sequences=True),
    SimpleRNN(1, return_sequences=True),
    SimpleRNN(1)
])
rnn3_model.summary()

In [ ]:
rnn3_model.compile(loss="mse", optimizer="adam")

In [ ]:
rnn3_hist = rnn3_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))